In [1]:
import pandas as pd
df = pd.read_csv("../../combined_file.csv")
df.drop(['definition', 'column_name'], axis=1, inplace=True)
df.columns

C:\Users\Franc\AppData\Local\Temp\ipykernel_12684\2358656560.py:2: DtypeWarning: Columns (0,1,2,8,9,12,13,15,17,18,19,21,22,23,24,25,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../combined_file.csv")


Index(['most_recent_observation', 'most_recent_observation_type',
       'common_name', 'scientific_name', 'city', 'state',
       'longitude_coordinate', 'latitude_coordinate', 'address', 'condition',
       'height_M', 'native', 'height_binned_M',
       'diameter_breast_height_binned_CM', 'greater_metro', 'city_ID',
       'tree_ID', 'planted_date', 'retired_date', 'location_type', 'zipcode',
       'neighborhood', 'location_name', 'ward', 'district', 'overhead_utility',
       'diameter_breast_height_CM', 'percent_population'],
      dtype='object')

In [2]:
df = df[["state", "city", "common_name"]]
df = df.dropna()
df

,state,city,common_name
0,New Mexico,Albuquerque,Locust
1,New Mexico,Albuquerque,Oak
2,New Mexico,Albuquerque,Ash
3,New Mexico,Albuquerque,Scotch pine
4,New Mexico,Albuquerque,Honeylocust
...,...,...,...
5642805,District of Columbia,Washington DC,Redmond american linden
5642806,District of Columbia,Washington DC,Dogwood
5642807,District of Columbia,Washington DC,Flowering dogwood
5642808,District of Columbia,Washington DC,(see notes)


In [3]:
df = df[df["common_name"]!="ok to plant vacant"]

In [4]:
# Count of unique cities per state
city_counts = df.groupby('state')['city'].nunique()
print(city_counts)

# Change to "Other" the states with less than 3 cities
states_to_rename = city_counts[city_counts < 3].index
df['state'] = df['state'].apply(lambda x: 'Other' if x in states_to_rename else x)

# Updated count of unique cities per state
updated_city_counts = df.groupby('state')['city'].nunique()
print(updated_city_counts)

state
Arizona                  1
California              13
Colorado                 3
District of Columbia     1
Florida                  7
Hawaii                   7
Iowa                    37
Kansas                   1
Kentucky                 1
Louisiana                1
Maryland                 1
Massachusetts            1
Michigan                 2
Missouri                 1
Nevada                   1
New Mexico               1
New York                 3
North Carolina           2
Ohio                     1
Oklahoma                 1
Oregon                   1
Pennsylvania             1
South Dakota             1
Tennessee                2
Texas                    4
Virginia                 1
Washington               1
Wisconsin               23
Name: city, dtype: int64


C:\Users\Franc\AppData\Local\Temp\ipykernel_12684\1695214080.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['state'] = df['state'].apply(lambda x: 'Other' if x in states_to_rename else x)


state
California    13
Colorado       3
Florida        7
Hawaii         7
Iowa          37
New York       3
Other         23
Texas          4
Wisconsin     23
Name: city, dtype: int64


In [5]:
df["state"].unique()

array(['Other', 'California', 'Colorado', 'Texas', 'New York', 'Florida',
       'Iowa', 'Hawaii', 'Wisconsin'], dtype=object)

In [6]:
# Top 30 most frequent trees per state
top_trees_by_state = df['common_name'].groupby([df['state'], df['common_name']]).size().reset_index(name='count')
top_trees_by_state = top_trees_by_state.groupby('state').apply(lambda x: x.nlargest(30, 'count')).reset_index(drop=True)

# Count trees per city (only consider the trees in the top 30 of the state of the city)
result = df.merge(top_trees_by_state, on=['state', 'common_name'], how='inner')
result = result.groupby(['state', 'city', 'common_name']).size().reset_index(name='count')

print(result)

           state           city          common_name  count
0     California        Anaheim    American sweetgum   1433
1     California        Anaheim          Bottle tree    416
2     California        Anaheim         Brisbane box   1609
3     California        Anaheim  California sycamore    634
4     California        Anaheim         Camphor tree   2567
...          ...            ...                  ...    ...
2014   Wisconsin  Whitefish Bay          Honeylocust     10
2015   Wisconsin  Whitefish Bay         Norway maple      2
2016   Wisconsin  Whitefish Bay              Red oak      7
2017   Wisconsin  Whitefish Bay          Sugar maple      1
2018   Wisconsin  Whitefish Bay            White ash      3

[2019 rows x 4 columns]


In [7]:
result.to_csv("../datasets/state_species_count.csv", index=False)